In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import autograd
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
minmax= MinMaxScaler()

images = np.load("images.npy")
labels = np.load("labels.npy")
HEIGHT = images.shape[1]
WIDTH = images.shape[2]
ELEMENT = HEIGHT*WIDTH

# images_processed = minmax.fit_transform(images.reshape(images.shape[0], ELEMENT))
# images_reshaped = images.reshape(images_processed.shape[0], 1, HEIGHT,WIDTH)
images_reshaped = images.reshape(images.shape[0], 1, HEIGHT,WIDTH)
# images_normed =  (images_reshaped - np.mean(images_reshaped, axis = 0))/np.std(images_reshaped, axis = 0)


NUM_IMAGES =  images_reshaped.shape[0]# number of images
NUM_VAL_IMAGES = NUM_IMAGES // 100
NUM_TRAIN_IMGAGES = NUM_IMAGES - NUM_VAL_IMAGES


indices = np.random.permutation(images_reshaped.shape[0])
training_idx, test_idx = indices[:NUM_TRAIN_IMGAGES], indices[NUM_TRAIN_IMGAGES:]
training_image, test_image = images_reshaped[training_idx,:], images_reshaped[test_idx,:]
training_label = labels[training_idx]
test_label = labels[test_idx]
class TooSimpleConvNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # 3x3 convolution that takes in an image with one channel
        # and outputs an image with 8 channels.
#         self.conv1 = torch.nn.Conv2d(1, 8, kernel_size=3)
        self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2) #Adddddddddddd
#         self.bn1 = torch.nn.BatchNorm2d(8) # addddded
        self.bn1 = torch.nn.BatchNorm2d(32)
        torch.nn.init.xavier_uniform_(self.conv1.weight)
        # 3x3 convolution that takes in an image with 8 channels
        # and outputs an image with 16 channels. The output image
        # has approximately half the height and half the width
        # because of the stride of 2.

        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2) #Addddddddddd
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        self.bn2 = torch.nn.BatchNorm2d(64)

        self.conv3 = torch.nn.Conv2d(64, 32, kernel_size=5, stride=1, padding=2) #Addddddddddd
        torch.nn.init.xavier_uniform_(self.conv3.weight)
        self.bn3 = torch.nn.BatchNorm2d(32)


        self.final_conv = torch.nn.Conv2d(32, 5, kernel_size=1) #addddddddddddddd
        torch.nn.init.xavier_uniform_(self.final_conv.weight)
    
    
        self.drop_out = torch.nn.Dropout(0.3) #Dropout 



    def forward(self, x):
        # Convolutions work with images of shape
        # [batch_size, num_channels, height, width]
        x = x.view(-1, HEIGHT, WIDTH).unsqueeze(1)

        ################
        x = self.bn1(self.conv1(x))
        x = F.relu(x)
        x = self.bn2(self.conv2(x))
        x = F.relu(x)
        x = self.bn3(self.conv3(x))
        x = F.relu(x)
        
        
        ###################
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         ###################
        x = self.drop_out(x)

        n, c, h, w = x.size()
        x = F.avg_pool2d(x, kernel_size=[h, w])

        x = self.final_conv(x).view(-1, NUM_CLASSES)
  
        return x

def approx_train_accuracy(y_hat, labels):
    labels = labels.detach().numpy()
    y_hat = y_hat.detach().numpy()

    return np.sum(y_hat==labels)/float(labels.size)

def train(batch_size, train_seqs, train_labels):
    # i is is a 1-D array with shape [batch_size]
    i = np.random.choice(train_seqs.shape[0], size=batch_size, replace=False)
    x = torch.from_numpy(train_seqs[i].astype(np.float32))
    y = torch.from_numpy(train_labels[i].astype(np.int))
    # Forward pass: Compute predicted y by passing x to the model
    y_hat_ = model(x)
    
    # Compute and print loss
    loss = F.cross_entropy(y_hat_, y)
    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()
    
model = TooSimpleConvNN()


# Loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.1)


# train_accs, val_accs = [], []
NUM_OPT_STEPS = 2000
batch_size = 10


# # ELEMENTS = images_reshaped.shape[1]# length of the image vector 
D_H = 100 # numbers of hidden layers 
NUM_CLASSES = 5 # number of classes
loss_store = []
train_accs, val_accs = [], []



for i in range(NUM_OPT_STEPS):
    scheduler.step()
    loss = train(batch_size,training_image,training_label )

    if i % 100 == 0:
        radom_index = np.random.choice(training_image.shape[0], size= 1000, replace=False)
        x_acc = torch.from_numpy(training_image[radom_index].astype(np.float32))
        y_acc = torch.from_numpy(training_label[radom_index].astype(np.int))
        y_hat_acc = model(x_acc)
        _, y_pred = torch.max(y_hat_acc,1)
        accuracy = np.sum((y_pred==y_acc).detach().numpy())/(y_acc.size())


In [6]:
images_test = np.load("test_images.npy")
# images_processed = minmax.fit_transform(images_test.reshape(images_test.shape[0], ELEMENT))
images_test_reshaped = images_test.reshape(images_test.shape[0], 1, HEIGHT,WIDTH)


print(images_test_reshaped.shape, images_test.shape)

# Test the model
model.eval()
with torch.no_grad():
    label = []
#     total = []
    images_val = images_test_reshaped
    
    for i in range(images_test_reshaped.shape[0]):
        x_val = torch.from_numpy(images_val[i].astype(np.float32))
#         labels_val = torch.from_numpy(labels_val.astype(np.int))

        outputs = model(x_val)
        predicted = torch.max(outputs, 1)[1]
        
        label.append(predicted.item())       


(5000, 1, 26, 26) (5000, 26, 26)


In [3]:
import pandas as pd    
raw_data = {'id': range(images_test_reshaped.shape[0]), 'label': label}
df = pd.DataFrame(raw_data, columns = ['id', 'label'])
df.to_csv('test_result.csv', sep=',', index=False)